In [1]:
import ee
ee.Initialize()

In [2]:
from iteru import *

In [3]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [4]:
startYear = 2020
startMonth =1
startDay = 1
endYear = 2022
endMonth = 3
endDay = 1

In [5]:
SAR,dates_sequences = S1_SAR_col(GERD_aoi,startYear,startMonth,startDay,endYear,endMonth,endDay)

In [6]:
def SAR_timeseries_url(col, aoi, vis_method='rgb', frame_per_second=2, crs='EPSG:3857', dimensions=900):

    try:
        if isinstance(col, ee.ImageCollection):
            pass
        else:
            raise Exception('The input should be ee.ImageCollection')

    except Exception as e:
        print(e)

    try:
        if isinstance(aoi, ee.Geometry):
            pass
        else:
            raise Exception('The input should be ee.Geometry')

    except Exception as e:
        print(e)

    palette = None
    vis_min = -25,
    vis_max = 5,
    
    try: 
        if vis_method == 'rgb':

            vis_min = [-25, -25, 0]
            vis_max = [0, 0, 5]
            bands = ['VV', 'VH', 'VV/VH']

        elif vis_method == 'single_band_VV':
            bands = ['VV']

        elif vis_method == 'single_band_VH':

            bands = ['VH']

        elif vis_method == 'water_mask_only':

            col = col.map(filterSpeckles).map(water_classify)
            bands = ['Water_mask']
            palette = ['00FFFF', '0000FF']

        elif vis_method == 'rgb_water_mosaic'  :

            col = col.map(filterSpeckles).map(water_classify).map(rgb_water_mosaic)

            bands = ['vis-red','vis-green','vis-blue']
            vis_min = 0
            vis_max = 255
        else:
            raise Exception('Invalid visualization method input')
            
    except Exception as e:
        print(e)
        return 
    else:
        
        videoArgs = {
            'dimensions': dimensions,
            'region': aoi,
            'framesPerSecond': frame_per_second,
            'crs': crs,
            'min': vis_min,
            'max': vis_max,
            'bands': bands,
            'palette' : palette

        }
#         url = col.getVideoThumbURL(videoArgs)
#         return url
        try:

            url = col.getVideoThumbURL(videoArgs)

            return url

        except Exception:

            print('The number of requested images exceeded the memory limit. Please, redcue the time interval')

            return


In [7]:
url = SAR_timeseries_url(SAR,GERD_aoi,vis_method='rgb_water_mosaic')

In [8]:
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/0fc1934dca9877c2cdcb91b0a2a91464-789a14f77281c91cd628baa99d16c335:getPixels


In [9]:
SAR = SAR.map(filterSpeckles).map(water_classify).map(rgb_water_mosaic)